In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2"
sys.path.append('/root/code')

from definitions import LOG_DIR, WEIGHT_DIR
import tensorflow as tf
from tensorflow import keras
import time
import datetime
import numpy as np
from utils.contrastiveLoss import ContrastiveLoss
from models.SCNN18 import SCNN18
from models.SCNN18_Sigmoid import SCNN18_Sigmoid
from models.SCNN18_random_aug import SCNN18_random_aug
from models.SCNN18_Sigmoid_Target_Task import SCNN18_Sigmoid_Target_Task
from models.SCNN18_Target_Task import SCNN18_Target_Task
import utils.dataset as dataset
import logging
from logging import handlers
from utils.voting import voting
from tensorflow.keras import layers as klayers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import Callback

In [2]:
LOG = logging.getLogger('root')

class TrainingStepLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        LOG.info(f'End of Epoch {epoch + 1} - Loss: {logs["loss"]:.4f}')

def initLog(debug=False):
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s %(levelname)s %(message)s',
        datefmt='%Y-%m-%d %H:%M',
        handlers=[logging.StreamHandler(), handlers.RotatingFileHandler('simclr_0.1s_audio.log', "w", 1024 * 1024 * 100, 3, "utf-8")]
    )
    LOG.setLevel(logging.DEBUG if debug else logging.INFO)
    tf.get_logger().setLevel('ERROR')

In [3]:
initLog()

In [4]:
def get_optimizer(optimizer, lr):
    optimizer = optimizer.lower()
    if optimizer == 'adadelta':
        return tf.optimizers.Adadelta() if lr == 0 else tf.optimizers.Adadelta(learning_rate=lr)
    elif optimizer == 'adagrad':
        return tf.optimizers.Adagrad() if lr == 0 else tf.optimizers.Adagrad(learning_rate=lr)
    elif optimizer == 'adam':
        return tf.optimizers.Adam() if lr == 0 else tf.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'adamax':
        return tf.optimizers.Adamax() if lr == 0 else tf.optimizers.Adamax(learning_rate=lr)
    elif optimizer == 'sgd':
        return tf.optimizers.SGD() if lr == 0 else tf.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop':
        return tf.optimizers.RMSprop() if lr == 0 else tf.optimizers.RMSprop(learning_rate=lr)

In [5]:
# Define model architecture & define hyper-parameter
lr = 1e-4
max_epochs = 15
sample_size=[32000, 1]
batch_size = 64
input_shape = tuple(sample_size)
output_shape = 10

In [6]:
class SaveWeightsCallback(Callback):
    def __init__(self, save_freq, max_epochs):
        super(SaveWeightsCallback, self).__init__()
        self.save_freq = save_freq
        self.max_epochs = max_epochs

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0 or (epoch + 1) == self.max_epochs:
            file_path = os.path.join(WEIGHT_DIR, f"{str(datetime.date.today())}_SCNN18_SimCLR_Sigmoid_down_stream_target_task_Jamendo_{epoch + 1}epochs_all_layer_train.h5")
            self.model.save_weights(file_path)
            print(f'Saved model weights at epoch {epoch + 1} to {file_path}')

In [7]:
# Down-stream train

loaded_weight = '2024-08-04_SCNN18_SimCLR_Sigmoid_down_stream_target_task_Jamendo_300epochs_last_layer_train.h5'

strategy = tf.distribute.MirroredStrategy(devices=[f'/gpu:{i}' for i in range(3)])
with strategy.scope():
    optimizer = get_optimizer('adam', lr)
    # Compile the model
    down_stream_model = SCNN18_Sigmoid_Target_Task(input_shape, output_shape).model()
    down_stream_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    # down_stream_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

down_stream_model.load_weights(os.path.join(WEIGHT_DIR, loaded_weight), by_name=True)
# for i, layer in enumerate(down_stream_model.layers):
#     if i < len(down_stream_model.layers) - 1:
#         down_stream_model.layers[i].trainable = False
#     else:
#         pass

down_stream_model.summary(print_fn=LOG.info)

save_freq = 3
save_weights_callback = SaveWeightsCallback(save_freq, max_epochs)

train_ds_path = './SCNN18_0.1second/SCNNR-Jamendo-train.h5'
train_ds = dataset.load(train_ds_path).batch(batch_size)

down_stream_model.fit(train_ds, epochs=max_epochs, callbacks=[save_weights_callback, TrainingStepLogger()])
# down_stream_model.save_weights(os.path.join(WEIGHT_DIR, f"{str(datetime.date.today())}_SCNN18_BYOL_Sigmoid_down_stream_target_task_Jamendo_300epochs_last_layer_train.h5"))

2024-08-06 07:44 INFO Model: "functional_1"
2024-08-06 07:44 INFO _________________________________________________________________
2024-08-06 07:44 INFO Layer (type)                 Output Shape              Param #   
2024-08-06 07:44 INFO =================================================================
2024-08-06 07:44 INFO input (InputLayer)           [(None, 32000, 1)]        0         
2024-08-06 07:44 INFO _________________________________________________________________
2024-08-06 07:44 INFO stsa (STSA)                  (None, 63, 1024, 1)       4194304   
2024-08-06 07:44 INFO _________________________________________________________________
2024-08-06 07:44 INFO random_zoom (RandomZoom)     (None, 63, 1024, 1)       0         
2024-08-06 07:44 INFO _________________________________________________________________
2024-08-06 07:44 INFO uscl_conv1 (USCLLayer)       (None, 21, 512, 64)       704       
2024-08-06 07:44 INFO ______________________________________________________

Epoch 1/15


KeyboardInterrupt: 